# OpenAI MuseNet Colab Notebook (ver 0.5)

***

### Source code is courtesy of https://github.com/MrCheeze/musenet-midi and https://github.com/daanklijn/musenet-midi-py

***

### Project Los Angeles
### Tegridy Code 2022

***

# Setup Environment

In [ ]:
#@title Install dependecies
!pip install requests
!pip install mido

In [ ]:
#@title Import modules

import secrets
import requests
from mido import Message, MidiFile, MidiTrack

# Querry the MuseNet API

In [ ]:
#@title Querry API

#@markdown Enter genre as a lowercase string or select it from it the menu
genre = "chopin" #@param ["chopin", "random"] {allow-input: true}

#@markdown Select instruments
piano = True #@param {type:"boolean"}
strings = True #@param {type:"boolean"}
winds = True #@param {type:"boolean"}
drums = True #@param {type:"boolean"}
harp = True #@param {type:"boolean"}
guitar = True #@param {type:"boolean"}
bass = True #@param {type:"boolean"}

#@markdown Generation settings
number_of_tokens_to_generate = 512 #@param {type:"slider", min:64, max:1024, step:8}
temperature = 0.8 #@param {type:"slider", min:0.1, max:2, step:0.1}
truncation = 20 #@param {type:"integer"}


genreList = ["chopin","mozart","rachmaninoff","ladygaga","country","disney","jazz","bach","beethoven","journey","thebeatles","video","broadway","franksinatra","bluegrass","tchaikovsky","liszt","everything","ragtime","andrehazes","cocciante","thecranberries","ligabue","metallica","traffic","philcollins","nineinchnails","thepretenders","sugarray","grandfunkrailroad","ron","ellington","fleetwoodmac","thebeachboys","kool & the gang","foreigner","tlc","scottjames","benfoldsfive","smashmouth","oasis","allsaints","donnasummer","weezer","bjork","mariahcarey","berte","cheaptrick","caroleking","thecars","gganderson","robertpalmer","zucchero","alicecooper","vanhalen","brucehornsby","coolio","jimmybuffett","lobo","badcompany","eminem","creedenceclearwaterrevival","deeppurple","shearinggeorge","robbiewilliams","dalla","ub40","lindaronstadt","sinatra","inxs","jonimitchell","michaeljackson","last","devo","shaniatwain","korn","brooksgarth","sweet","thewho","roxette","bowiedavid","beegees","renefroger","mina","estefangloria","mccartney","theventures","carboni","simplyred","santana","jewel","meatloaf","giorgia","nofx","rickymartin","thecure","thetemptations","tozzi","beck","eiffel65","jenniferlopez","reelbigfish","patsycline","richardcliff","styx","acdc","brucespringsteen","michaelgeorge","blondie","pinkfloyd","oldfieldmike","redhotchilipeppers","therollingstones","morandi","heart","robertaflack","pantera","alabama","jethrotull","hanson","mosch","ludwigvanbeethoven","dvorak","chrisrea","guns n' roses","duranduran","ericclapton","bettemidler","bwitched","gordonlightfoot","thegrassroots","chicago","whitezombie","michaelbolton","paulsimon","marillion","thepointersisters","theanimals","cher","haydn","aerosmith","supertramp","littleriverband","america","tonyorlando","tompetty","thecorrs","aliceinchains","kiss","prince","toto","vanmorrison","wagner","cashjohnny","annielennox","enya","thedoobiebrothers","thetragicallyhip","rush","laurapausini","stevemillerband","simonandgarfunkel","fiorellamannoia","olivianewton-john","carlysimon","elvispresley","vangelis","bobdylan","bbking","vengaboys","paoli","thehollies","alainsouchon","pooh","raf","fiorello","lionelrichie","jimihendrix","theeverlybrothers","limpbizkit","donhenley","georgeharrison","threedognight","johnmellencamp","carpenters","raycharles","basie","billyocean","scorpions","royorbison","whitneyhouston","ironmaiden","jovanotti","alanjackson","barrymanilow","hueylewis","kennyloggins","chopinfrederic","talkingheads","themonkees","rem","jeanmicheljarre","michelezarrillo","eurythmics","thedoors","guesswho","miller","thefourseasons","matiabazar","tompettyandtheheartbreakers","chickcorea","scottjoplin","amedeominghi","bryanadams","paulaabdul","rossivasco","billyjoel","daniele","claudedebussy","gilbert & sullivan","chakakhan","nirvana","garbage","andreabocelli","johnnyrivers","emerson, lake & palmer","theallmanbrothersband","zappa","boston","mango","barbrastreisand","willsmith","ozzyosbourne","janetjackson","antonellovenditti","u2","humperdinckengelbert","jamiroquai","zero","chuckberry","spicegirls","ledzeppelin","masini","thekinks","eagles","billyidol","alanismorissette","joecocker","jimcroce","bobmarley","blacksabbath","stonetemplepilots","silverchair","paulmccartney","blur","nek","greenday","thepolice","depechemode","rageagainstthemachine","madonna","rogerskenny","brooks & dunn","883","thedrifters","amygrant","herman","toriamos","eltonjohn","britneyspears","lennykravitz","celentano","ringostarr","neildiamond","aqua","oscarpeterson","joejackson","moby","collinsphil","leosayer","takethat","electriclightorchestra","pearljam","marcanthony","borodin","petshopboys","stevienicks","hollybuddy","turnertina","annaoxa","zztop","sting","themoodyblues","ruggeri","creed","claudebolling","renzoarbore","erasure","elviscostello","airsupply","tinaturner","leali","petergabriel","nodoubt","bread","huey lewis & the news","brandy","level42","radiohead","georgebenson","wonderstevie","thesmashingpumpkins","cyndilauper","rodstewart","bush","ramazzotti","bobseger","theshadows","gershwin","cream","biagioantonacci","steviewonder","nomadi","direstraits","davidbowie","amostori","thealanparsonsproject","johnlennon","crosbystillsnashandyoung","battiato","kansas","clementi","richielionel","yes","brassensgeorges","steelydan","jacksonmichael","buddyholly","earthwindandfire","natkingcole","therascals","bonjovi","alanparsons","backstreetboys","glencampbell","howardcarpendale","thesupremes","villagepeople","blink-182","jacksonbrowne","sade","lynyrdskynyrd","foofighters","2unlimited","battisti","hall & oates","stansfieldlisa","genesis","boyzone","theoffspring","tomjones","davematthewsband","johnelton","neilyoung","dionnewarwick","aceofbase","marilynmanson","taylorjames","rkelly","grandi","sublime","edvardgrieg","tool","bachjohannsebastian","patbenatar","celinedion","queen","soundgarden","abba","drdre","defleppard","dominofats","realmccoy","natalieimbruglia","hole","spinners","arethafranklin","reospeedwagon","indian","movie","scottish","irish","african","taylorswift","shakira","blues","latin","katyperry","world","kpop","africandrum","michaelbuble","rihanna","gospel","beyonce","chinese","arabic","adele","kellyclarkson","theeagles","handel","rachmaninov","schumann","christmas","dance","punk","natl_anthem","brahms","rap","ravel","burgmueller","other","schubert","granados","albeniz","mendelssohn","debussy","grieg","moszkowski","godowsky","folk","mussorgsky","kids","balakirev","hymns","verdi","hummel","deleted","delibes","saint-saens","puccini","satie","offenbach","widor","songs","stravinsky","vivaldi","gurlitt","alkan","weber","strauss","traditional","rossini","mahler","soler","sousa","telemann","busoni","scarlatti","stamitz","classical","jstrauss2","gabrieli","nielsen","purcell","donizetti","kuhlau","gounod","gibbons","weiss","faure","holst","spohr","monteverdi","reger","bizet","elgar","czerny","sullivan","shostakovich","franck","rubinstein","albrechtsberger","paganini","diabelli","gottschalk","wieniawski","lully","morley","sibelius","scriabin","heller","thalberg","dowland","carulli","pachelbel","sor","marcello","ketterer","rimsky-korsakov","ascher","bruckner","janequin","anonymous","kreutzer","sanz","joplin","susato","giuliani","lassus","palestrina","smetana","berlioz","couperin","gomolka","daquin","herz","campion","walthew","pergolesi","reicha","polak","holborne","hassler","corelli","cato","azzaiolo","anerio","gastoldi","goudimel","dussek","prez","cimarosa","byrd","praetorius","rameau","khachaturian","machaut","gade","perosi","gorzanis","smith","haberbier","carr","marais","glazunov","guerrero","cabanilles","losy","roman","hasse","sammartini","blow","zipoli","duvernoy","aguado","cherubini","victoria","field","andersen","poulenc","d'aragona","lemire","krakowa","maier","rimini","encina","banchieri","best","galilei","warhorse","gypsy","soundtrack","encore","roblaidlow","nationalanthems","benjyshelton","ongcmu","crosbystillsnashyoung","smashingpumpkins","aaaaaaaaaaa","alanismorrisette","animenz","onedirection","nintendo","disneythemes","gunsnroses","rollingstones","juliancasablancas","abdelmoinealfa","berckmansdeoliveira","moviethemes","beachboys","davemathews","videogamethemes","moabberckmansdeoliveira","unknown","cameronleesimpson","johannsebastianbach","thecarpenters","elo","nightwish","blink182","emersonlakeandpalmer","tvthemes"]
if genre == 'random':
	genre = genreList[secrets.randbelow(len(genreList))]

print('Starting up...')

headers = {"Content-Type": "application/json"}

data = json.dumps({
		
				"genre": "chopin",

				"instrument":{
					"piano": piano,
					"strings": strings,
					"winds": winds,
					"drums": drums,
					"harp": harp,
					"guitar": guitar,
					"bass": bass
				},

			"encoding": "",

			"temperature": temperature,

			"truncation": truncation,

			"generationLength": number_of_tokens_to_generate,

			"audioFormat": "audio/ogg"})

print('Requesting data from the API. Please wait...')
response = requests.post('https://musenet.openai.com/sample', headers=headers, data=data)

print('Decoding...')
res = response.json()

encoding = res['completions'][0]['encoding']

print('Done!')

# Convert MuseNet output to MIDI

In [ ]:
#@title Convert to MIDI
INSTRUMENTS = ["piano", "piano", "piano", "piano", "piano", "piano", "piano", "piano", "piano",
               "piano", "piano", "piano", "piano", "piano",
               "violin", "violin", "cello", "cello", "bass", "bass", "guitar", "guitar",
               "flute", "flute", "clarinet", "clarinet", "trumpet", "trumpet", "harp", "harp"]

TRACK_INDEX = {"piano": 0, "violin": 1, "cello": 2, "bass": 3, "guitar": 4, "flute": 5,
               "clarinet": 6, "trumpet": 7, "harp": 8, "drum": 9}

VOLUMES = [0, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 80, 0, 80, 0, 80, 0, 80, 0, 80,
           0, 80, 0, 80, 0, 80, 0]

DRUM_VELOCITY = 80

DELAY_MULTIPLIER = 10


def parse_token(token: int):
    if 0 <= token < 3840:
        note = token % 128
        inst_vol_index = token >> 7
        velocity = VOLUMES[inst_vol_index]
        instrument = INSTRUMENTS[inst_vol_index]
        track = instrument
        return {"type": "note", "note": note, "channel": track, "velocity": velocity}
    elif 3840 <= token < 3968:
        note = token % 128
        return {"type": "note", "note": note, "channel": "drum", "velocity": DRUM_VELOCITY}
    elif 3968 <= token < 4096:
        delay = (token % 128) + 1
        return {"type": "wait", "time": delay}
    elif token == 4096:
        return {"type": "start"}
    else:
        return {"type": "invalid"}


midi = MidiFile()
tracks = [MidiTrack() for _ in range(10)]
delta_times = [0 for _ in range(10)]
for token in encoding.split():
    parsed = parse_token(int(token))
    typez = parsed['type']
    if typez == 'note':
        track_index = parsed['channel']
        volume = parsed['velocity']
        #print(delta_times)
        #print(track_index)
        delay = delta_times[TRACK_INDEX[track_index]]
        if delay > 0:
            parsed['time'] = delta_times[TRACK_INDEX[track_index]] * DELAY_MULTIPLIER
        parsed['channel'] = TRACK_INDEX[track_index]
        parsed['type'] = "note_on" if volume > 0 else "note_off"
        delta_times[TRACK_INDEX[track_index]] = 0

        track = tracks[TRACK_INDEX[track_index]]
        track.append(Message(**parsed))
    elif typez == 'wait':
        for j in range(10):
            delta_times[j] += parsed['time']

non_empty_tracks = [track for track in tracks if len(track) > 2]
#print(tracks)
midi.tracks.extend(non_empty_tracks)
midi.save('/content/MuseNet-Output.mid')

# Congrats! You did it! :)